## [N으로 표현](https://school.programmers.co.kr/learn/courses/30/lessons/42895)
- 주어진 N만으로 number를 표현하기 위한 N의 최소 사용 횟수를 반환하시오
- 예를 들어 5로 12를 표현하는 최소 연산은 (55 + 5) / 5 라서 4이다.(5가 4번 쓰였으니까)
- 단, 나머지 연산은 `//`로 하며 최솟값이 8보다 크면 -1을 리턴한다.
- 수식은 괄호와 사칙연산만 사용할 수 있다.
- 3시간 35분! 미친!

In [5]:
N = 5
number = 31168

In [6]:
from collections import defaultdict, deque

# hpq = [] # 최소 우선순위 힙
dq = deque([])

# dp를 dct로 구성해도 될 듯? : 리스트 구성보다 좋은 점은, 32000 * 2 개의 리스트 전부를 탐색할 필요가 없을 듯
# dct의 key = 숫자, value = N이 사용된 최소 갯수
dp = defaultdict(lambda : 9)

# 1. N으로 만들 수 있는 값들부터 dp에 넣고, 힙에는 순서를 바꿔서 넣는다
for i in range(1, 6):
    init_N = int(str(N) * (i))
    if init_N > 32000: # number는 1 ~ 32000
        break
    dp[init_N] = i
    dq.append((i, init_N))

# 2. 힙에서 값을 하나씩 뺀 다음, dp에 있는 값들에 대해 사칙연산을 적용한다
# while문은 A (연산) B 에서 A를 정하는 과정임
while dq:

    N_count1, value1 = dq.popleft() # N 사용횟수, 값

    if N_count1 >= dp[number]: 
        continue

    # dq에 있던 도중 dp에 있는 해당 값을 만드는 N 숫자가 더 작아졌을 수 있다
    if dp[value1] > N_count1: 
        continue 

    if value1 == 0: # 0은 세는 의미가 없다 111
        continue
    
    # dp에 있는 값들에 대해 사칙연산을 해서, 더 적은 N이 쓰인 경우 dq에 추가한다.
    temp_dp = dp.copy() # dictionary changed size during iteration 오류 : 반복문중 dict의 크기가 바뀌는 오류

    # 현재 dp에 있는 값들만 계산한다 : 즉, 아래 반복문으로 새로 얻은 값은 지금 계산 안한다

    for value2, N_count2 in temp_dp.items():
        
        if value2 == 0 : # 0은 세는 의미가 없다 222
            continue

        next_count = N_count1 + N_count2 # 연산된 값은 두 N이 사용된 횟수의 합이다

        if next_count > 8: # 8보다 큰 경우는 셀 필요 없음
            continue

        plus_value = value1 + value2
        
        minus_value1 = value1 - value2
        minus_value2 = value2 - value1

        multiply_value = value1 * value2
        
        divide_value1 = value1 // value2
        divide_value2 = value2 // value1

        # dp의 값에는 들어간 N의 갯수가 저장된다
        for i in [plus_value, minus_value1, minus_value2, multiply_value, divide_value1, divide_value2]:

            if dp[i] > next_count:
                dp[i] = next_count
                dq.append((dp[i], i))


print(dp[number] if dp[number] <= 8 else - 1)
# return dp[number] if dp[number] <= 8 else -1

-1


## 풀기 전 생각

### 1. DP
```
number의 범위가 32000 이하임
1) 1부터 32000까지의 길이를 갖는 리스트를 만든다
2) N으로 구성할 수 있는 값들에 N이 몇 번 쓰였는지 넣는다(5면 5에 1, 55에 2, 555에 3, 5555에 4...)
3) 최소 우선순위 힙으로 구성함 : 그래서 힙은 (N 사용 횟수, 인덱스(=값))으로 구성됨
```
### 2. 연산을 어떻게 구성할 것인가
```
DP에 들어가 있는 값들에 대해 (자기 자신 포함) 사칙연산을 가할 수 있음
이 때, 자기 항은 앞에 오고 다른 항은 뒤에 가게 구성할 수도 있음
# 그런데 DP 계산 과정에서 마이너스 값이 포함될 수도 있음 
예를 들어 5로 3을 만든다고 하면, ((5 - 55) // 5) // 5 + 5란 말임? 여기서 -50, -10, -2에 사용된 N의 수가 있어야 하지 않나?
그러면 마이너스 인덱스를 어디까지 만들어야 하지? -> `dict`로 구성하면 간단하게 해결됨
```

## 푸는 중 생각
- 구현 방식은 최초에 5로만 구현한다 -> 이것들을 갖고 새로운 숫자를 계산한다 -> 사용된 5의 개수를 값으로, 이걸로 계산된 값을 키로 하는 dct를 만든다

1. **언제 종료할 수 있을까?** 를 생각해보자
- 일단 문제에서는 `8보다 큰 경우를 -1로 한다`와, `number 값의 크기`를 정한 상태임
- `number 값의 크기`를 이용할 수 있을 것 같은데, 상한을 어디로 정하냐가 문제다.

---
### 2. 라고 할 수 있을까요?
- 최소 우선순위 큐를 이용해서 가장 적은 N부터 계산하는 방식을 쓰고 있다. -> 가장 먼저 `number`를 갱신하는 상황에서의 N의 개수가 정답이라고 가정했다
- 왼쪽 값을 그렇게 가져갔다면, 오른쪽 값도 그렇게 가져가야 한다. 즉 heapq을 두번 써야 함
- 왜냐하면 지금까지의 코드에서 오른쪽 값은 그냥 `dp`에 있는 모든 값들을 가져왔기 때문이다.

### 3. 결국, 모든 경우의 수를 돌리되 제외하는 것들이 있어야 할 것 같음
- 예를 들어서 N = 1과 N = 6으로 만든 값이 있다고 치자
- 근데 알고보니까 뒤에서 N = 2와 N = 3으로 만들 수 있었던 거임!
- 근데 기존 구현(가장 먼저 나온 값을 이용)은 (1, 6)으로 만들자마자 끝내버리는 방식임
- 따라서 (효율적인) 모든 경우의 수를 탐색해야 하고, 따라서 정렬이 필요 없음 - 그래서 `heapq` 대신 `queue`를 쓰겠음

### 4. 제외할 값들
1. 음수 탈출 방법
- 음수에서 음수를 뺀다
- 음수 값을 나누고 더한다


In [ ]:
# 5 55
# target : 13
# 